# Step Count From Video

### Overview
Count the number of steps from a carefully controlled treadmill video. We identify a unique color in a region of interest (ROI), such as a shoelace. This color is tracked and the number of zero crossing (normalized) is correlated to a step. Specifically create an object around the the color of interest and track the trajectory in Euclidean space (x,y). Zero crossings are estimated by identifing the zero slope and inflection point of the y coordinate time series.

### Improvements
- <font color='red'>Automate ROI (DONE)</font> 
- Physically place unique color tags (neon stickers) on shoes during test-
- Automate start and stop frames
- Consider contours in Visualization
- Test accuracy over various test scenarios
- Consider transforming to HSV space to mitigate shadows
- Automate setting of kernel size (for object detect) and mask threshold
- Use history to track trajectory
- <font color='red'>Minimum stride rate (DONE)</font>

### Imports and System Parameters

In [2]:
import sys
import cv2
import matplotlib.pyplot as plt
import numpy as np
%matplotlib osx
print('Using OpenCV version %s'%cv2.__version__)
print('Using Numpy version  %s'%np.__version__)
ver = sys.version.split('|')
print('Using python verion  %s'%ver[0])
print('('+ver[1]+')')
print(ver[2])

# system parameters
# Display tracking Video and Debug Messages
#DISP = True
DISP = True

# size of kernel
kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (8,8))

# min blob size
minBlobArea = 5

# +/- in RGB color space
offset = 20

# file to interrogate
filename = './Downloads/StepCountRateVideos/20161110_152638 KeDe Step Count.mp4'

# frame window over which actual steps are taken
startFrame = 500
endFrame = 25700

Using OpenCV version 3.1.0
Using Numpy version  1.11.1
Using python verion  3.5.2 
(Anaconda 4.2.0 (x86_64))
 (default, Jul  2 2016, 17:52:12) 
[GCC 4.2.1 Compatible Apple LLVM 4.2 (clang-425.0.28)]


### Function Definitions

In [4]:
def onclick(event):
    print('button=%d, x=%d, y=%d, xdata=%f, ydata=%f' %
          (event.button, event.x, event.y, event.xdata, event.ydata))
    global idxList
    idxList.append([event.xdata, event.ydata])
#    global axRef
#    axRef.scatter(int(event.xdata), int(event.ydata))
    
def getPoint(x,y,img):
    temp = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    return temp[x,y]

def getLineSegment(x1,x2,y,img):
    temp = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    return temp[x1:x2,y]

def colorFromIdx(idx, frame):
    x,y = idx 
    return frame[int(y), int(x), :]

def plotMask(idx, frameColor, frame, offset):
    idColor = colorFromIdx(idx, frameColor)
    mask = np.zeros(frame.shape)
    for ii,color in enumerate(idColor):
        mask[:,:,ii] = (frame[:,:,ii] > color-offset) & (frame[:,:,ii]<color+offset)

    plt.figure()
    plt.imshow(np.sum(mask,axis=2))
    plt.title(str(idx))

def plotRefPoint(idx, ax):
    x,y = idx 
    ax.scatter(int(x), int(y))
    
def getMask(idColor, frame, offset):
    mask = np.zeros(frame.shape)
    for ii,color in enumerate(idColor):
        mask[:,:,ii] = (frame[:,:,ii] > color-offset) & (frame[:,:,ii]<color+offset) 
    mask = np.sum(mask, axis=2) ==3
    return mask

def getBlob(mask):
    blob = cv2.dilate(mask.astype(np.uint8), kernel)
    return blob

def getCentroid(blob):
    temp = blob.copy()
    temp[temp==0] = 255
    temp[temp==1] = 0
    keypoints = detector.detect(temp)
#    keypoints = detector.detect(blob)
    return keypoints


# Set up the SimpleBlobdetector with default parameters.
params = cv2.SimpleBlobDetector_Params()

# Change thresholds
params.minThreshold = 0;
params.maxThreshold = 256;

# Filter by Area.
params.filterByArea = True
params.minArea = minBlobArea

# Filter by Circularity
params.filterByCircularity = False
params.minCircularity = 0.1

# Filter by Convexity
params.filterByConvexity = False
params.minConvexity = 0.5

# Filter by Inertia
params.filterByInertia = False
params.minInertiaRatio = 0.5

detector = cv2.SimpleBlobDetector_create(params)

def trackSteps(idx, roiRef, DISP, filename, roiPoints):
    
    x,y = idx
    r,c,h,w = roiPoints 
    idColor = colorFromIdx(idx, roiRef)
    xBuff = [x]
    yBuff = [y]
    frameSkip = 0

    cap = cv2.VideoCapture(filename)
    cap.set(cv2.CAP_PROP_POS_FRAMES,frameSkip)
    ret,frame = cap.read()
    count = frameSkip

    while ret:
        count = count + 1
        roi = frame[r:h,c:w]

        mask = getMask(idColor, roi, offset)
        blob = getBlob(mask)
        keypoints = getCentroid(blob)

        # if we find any blobs
        if keypoints:

            # assume one and use that as the centroid
            x,y = keypoints[0].pt

            # if more than one find the one closest to the last one
            minVal = float('inf')
            for kp in keypoints:
                x,y = kp.pt
                dist = np.power((x-xBuff[-1]),2)+np.power((y-yBuff[-1]),2)
                if dist < minVal:
                    minVal = dist
                    xOut = x
                    yout = y


        else:

            # if no blob found then use the old value
            # mostly we get here because the blob touches the edge of the image
            x = xBuff[-1]
            y = yBuff[-1]
            #input('')

        xBuff.append(x)
        yBuff.append(y)
           
        if DISP and (count%1000 == 0):
            print(count)

        if DISP:

            fr,fc,t = frame.shape
            frameMask = np.ones([fr,fc],dtype=np.uint8)      
            frameMask[r:h,c:w] = cv2.bitwise_xor(blob,1)        
            new = cv2.circle(cv2.bitwise_and(frame, frame, mask=frameMask), (int(x)+c,int(y)+r), 5, 255, -1)
            cv2.imshow('frame',new)

    #         if cv2.waitKey(1) & 0xFF == ord('q'):
    #             break

            k = cv2.waitKey(0)
            if k == 27:         # wait for ESC key to exit
                cv2.destroyAllWindows()
                break
            elif k == ord('n'): # wait for 's' for next frame
                t = 1

        ret,frame = cap.read()


    cap.release()
    return [xBuff, yBuff]

def trackPoints(filename, frameOffset, roiPoints):
    r,h,c,w = roiPoints 

    # grab and display a frame during run
    cap = cv2.VideoCapture(filename)
    cap.set(cv2.CAP_PROP_POS_FRAMES,frameOffset)
    fs = cap.get(cv2.CAP_PROP_FPS)

    ret,frame = cap.read()
    # set up the ROI for tracking
    roiRef = frame[r:r+h, c:c+w]
           
    cv2.namedWindow('select track points')
    cv2.setMouseCallback('select track points',draw_circle)

    while(1):
        cv2.imshow('image',roiRef)
        k = cv2.waitKey(1) & 0xFF
        if k == ord('m'):
            mode = not mode
        elif k == 27:
            break

    cv2.destroyAllWindows()

 
    # mouse click on columns to histogram
    return [roiRef, fs]

def countSteps(yBuff, startFrame, endFrame):
    # user input to figure out true offset
    threshStepSamples = 4 #min number of sample per stride based on max step rate

    # low pass filter to smooth
    N=10
    fc = fs/600.

    import scipy.signal as sp

    # use firwin with arbitrary Fs
    h=sp.firwin( numtaps=N, cutoff=fc, nyq=fs/2)
    filtY=sp.lfilter( h, 1.0, yBuff) # 'x' is the time-series data you are filtering


    # count number of positive inflextions points (first derivative zero / second neg)
    dy = np.diff(filtY)
    dy[dy>0] = 1
    dy[dy<0] = -1
    ddy = np.diff(dy)

    # this counts just one legg steps
    #ddy[ddy>0] = 0

    # if we assume a backswing == a forward swing with the other leg
    stepsToCount = ddy[startFrame:endFrame]
    totalSteps = np.count_nonzero(stepsToCount)
    print('Total Steps before minimum correction = %d'%totalSteps)

    A = np.diff(np.nonzero(stepsToCount))[0]
    newTotalSteps = len(A[A>threshStepSamples])
    print('Total Steps after minimum correction = %d'%newTotalSteps)
    return [filtY, dy, ddy, A]    

### Manually select unique color pixel to track in ROI

In [3]:
idxList = []
roiList = []

def draw_circle(event,x,y,flags,param):
    global idxList
    if event == cv2.EVENT_LBUTTONDOWN:
        cv2.circle(roiScale,(x,y),3,(255,0,0),-1)
        idxList.append([x,y])

def draw_roi(event,x,y,flags,param):
    global roiList
    if event == cv2.EVENT_LBUTTONDOWN:
        cv2.circle(frame,(x,y),3,(255,0,0),-1)
        print(x,y)
        roiList.append([x,y])     
        
        
# automate this with user input based on time maybe
frameOffset = 0

        
# grab and display a frame during run
cap = cv2.VideoCapture(filename)
cap.set(cv2.CAP_PROP_POS_FRAMES,frameOffset)
fs = cap.get(cv2.CAP_PROP_FPS)

ret,frame = cap.read()

cv2.namedWindow('select roi')
cv2.setMouseCallback('select roi',draw_roi)
while(1):
    cv2.imshow('select roi', frame)
    k = cv2.waitKey(1) & 0xFF
    if k == 27:
        break
cv2.destroyAllWindows() 

        
c,r = roiList[0]
w,h = roiList[1]
roiPoints = (r,c,h,w)

# set up the ROI for tracking
roi = frame[r:h,c:w]

cv2.namedWindow('select track points')
cv2.setMouseCallback('select track points',draw_circle)
roiScale = cv2.resize(roi, None,fx=2, fy=2, interpolation = cv2.INTER_CUBIC)
roiPristine = roiScale.copy()

while(1):
    cv2.imshow('select track points', roiScale)
    k = cv2.waitKey(1) & 0xFF
    if k == ord('m'):
        mode = not mode
    elif k == 27:
        break

cv2.destroyAllWindows() 

True

180 302
321 447


### Visualize selected Points

In [15]:
for idx in idxList:    
    plotMask(idx, roiPristine, frame, offset)

### Track 

In [16]:
superBuff = []
for ii,idx in enumerate(idxList):
    superBuff.append(trackSteps(idx, roiPristine, False, filename, roiPoints))


In [17]:
import pickle
with open(filename[:-4]+'_buffFile1.pkl','wb') as f:
    pickle.dump(superBuff,f)


 ### Filter and count steps

In [18]:
superFilter = []
startFrame = 670
endFrame = 25700

for buff in superBuff:
    xBuff,yBuff = buff
    superFilter.append(countSteps(yBuff, startFrame, endFrame))


Total Steps before minimum correction = 3237
Total Steps after minimum correction = 2545
Total Steps before minimum correction = 3252
Total Steps after minimum correction = 2329
Total Steps before minimum correction = 3126
Total Steps after minimum correction = 2133
Total Steps before minimum correction = 3810
Total Steps after minimum correction = 2632


#### Debug Code

In [20]:
plt.figure()
# plt.plot(filtY)
#plt.plot(filtY[20000:])
plt.plot(filtY,'.-')



NameError: name 'filtY' is not defined

In [19]:
%run 'import_plots.ipynb'

In [ ]:
ipplot(ddy)

In [ ]:
ipplot(smooth.tolist())

In [ ]:
B = [len(A[A>thresh]) for thresh in np.arange(20)]
pplot(B);

In [ ]:
np.where(A>)

In [ ]:
pplot(ddy[1:100])

In [ ]:
np.arange(len(yBuff))/fs

In [ ]:
fudge  = (len(xBuff)/fs) / 878.

In [6]:
fs=18

In [7]:
import pickle
S = pickle.load(open(filename[:-4]+'_buffFile1.pkl', 'rb'))
superFilter = []
# startFrame = 500
# endFrame = 25700

for buff in S:
    xBuff,yBuff = buff
    superFilter.append(countSteps(yBuff, startFrame, endFrame))

%run 'import_plots.ipynb'
%matplotlib osx

sns.set_style("white")
sns.set_style("ticks")
fig, ax = plt.subplots(figsize=(11, 6))

for filt in superFilter:
    smooth, slope, inflection, steprate = filt
#     B = [len(steprate[steprate>thresh]) for thresh in np.arange(20)]
#     plt.plot(B,marker='.')
    plt.plot(smooth)
plt.ylabel('Number Step', fontsize=14)
plt.tick_params(axis='y', labelsize=12)
plt.tick_params(which='major', axis='x', labelsize=12)
plt.grid(axis='y',color='grey', linestyle='--', lw=0.5, alpha=0.5)
plt.grid(axis='x',color='grey', linestyle='--', lw=0.5, alpha=0.5)
sns.despine(trim=True, left=True, bottom=True)

Total Steps before minimum correction = 3251
Total Steps after minimum correction = 2557
Total Steps before minimum correction = 3267
Total Steps after minimum correction = 2340
Total Steps before minimum correction = 3140
Total Steps after minimum correction = 2144
Total Steps before minimum correction = 3831
Total Steps after minimum correction = 2644


In [ ]:
import pickle
S = pickle.load(open('buffFile1', 'rb'))
superFilter = []
# startFrame = 500
# endFrame = 25700

for buff in S:
    xBuff,yBuff = buff
    superFilter.append(countSteps(yBuff, startFrame, endFrame))

%run 'import_plots.ipynb'
%matplotlib notebook

sns.set_style("white")
sns.set_style("ticks")
fig, ax = plt.subplots(figsize=(11, 6))

for filt in superFilter:
    smooth, slope, inflection, steprate = filt
    plt.hist(steprate, 100)

plt.ylabel('Number Step', fontsize=14)
plt.tick_params(axis='y', labelsize=12)
plt.tick_params(which='major', axis='x', labelsize=12)
plt.grid(axis='y',color='grey', linestyle='--', lw=0.5, alpha=0.5)
plt.grid(axis='x',color='grey', linestyle='--', lw=0.5, alpha=0.5)
sns.despine(trim=True, left=True, bottom=True)